In [ ]:
using CMBLensing
using PythonPlot


Cℓ  = camb(r=0.05, ℓmax=16000)         
Cℓn = noiseCℓs(μKarcminT=1, ℓknee=100)


θpix  = 1        # pixel size in arcmin
Nside = 128      # number of pixels per side in the map
pol   = :I       # type of data to use (can be :T, :P, or :TP)
T     = Float32  # data type (Float32 is ~2 as fast as Float64);
bandpass_mask = LowPass(5000);

(;f, f̃, ϕ, ds) = load_sim(
    seed = 3,
    Cℓ = Cℓ,
    Cℓn = Cℓn,
    θpix = θpix,
    T = T,
    Nside = Nside,
    beamFWHM = 1.4,
    pol = pol,
    bandpass_mask = bandpass_mask, 
    pixel_mask_kwargs = (edge_padding_deg=0, apodization_deg=0, num_ptsrcs=0),
)

┌ Error: LLVM extensions library unavailable for your platform:
│   aarch64-apple-darwin-libgfortran5-cxx11-llvm_version+18-julia_version+1.12.1
│ LLVM.jl will not be functional.
│ 
│ If you are using a custom version of LLVM, try building a
│ custom version of LLVMExtra_jll using `deps/build_local.jl`
└ @ LLVM ~/.julia/packages/LLVM/bzSzE/src/LLVM.jl:103


In [3]:
ds = ds(;)

(;Cf, Cϕ) = ds

BaseDataSet(d, Cf, Cn, Cn̂, M, M̂, B, B̂, logprior, Cϕ, Cf̃, D, G, L, Nϕ)

In [ ]:
using CMBLensing: Map, Loess

# inject loess into CMBLensing namespace
Base.eval(CMBLensing, :(loess = Loess.loess))



Ωstart = FieldTuple(ϕ = Map(zero(diag(ds.Cϕ)))) 

result = MAP_joint(
    (;),            # θ = empty parameter block
    ds,
    Ωstart;
    nsteps = 20,
    nburnin_update_hessian = 0,
    progress = true,
    conjgrad_kwargs = (tol=1e-3, nsteps=800),
)

fJ  = result.f
ϕJ  = result.ϕ
hist = result.history


MAP_joint:   0%|                                        |  ETA: N/A

In [ ]:

# ------------------------------------------------------------------
# 5) Compare spectra
# ------------------------------------------------------------------
ℓ = Cℓ.total.ℓ    # This prints the ell grid of the theoretical Cℓ
println("ℓ range in theory: ", extrema(ℓ))

loglog(ℓ.^4 .* Cℓ.total.ϕϕ, "k")                 # theory
loglog(get_ℓ⁴Cℓ(ϕ))                               # true phi
loglog(get_ℓ⁴Cℓ(ϕJ))                              # MAP reconstruction
loglog(get_ℓ⁴Cℓ(ϕJ - ϕ))                          # residual
loglog(get_ℓ⁴Cℓ(quadratic_estimate(ds).ϕqe))      # QE
xlim(100, 10000)
ylim(5e-12, 2e-6)
legend(["theory", raw"true $\phi$", raw"best-fit $\phi$",
        "difference", raw"QE $\phi$"])
xlabel(raw"$\ell$")
ylabel(raw"$\ell^4 C_\ell$")

PythonPlot.show()

# ------------------------------------------------------------------
# 5) Lensing map visualisation
# ------------------------------------------------------------------
PythonPlot.plot(
    1e6 .* [ϕ  ϕJ],
    title = ["true" "best-fit"] .* raw" $\phi$",
    vlim = 17
)


# ------------------------------------------------------------------
# 6) Correlation coefficient vs ell (ρℓ)
# ------------------------------------------------------------------
semilogx(get_ρℓ(ϕ, ϕJ))
semilogx(get_ρℓ(quadratic_estimate(ds).ϕqe, ϕ))
legend(["Joint MAP", "QE"])